##Packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# data imports
import pandas as pd
import numpy as np
from plotnine import *

# modeling imports
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures #Z-score variables, Polynomial
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error, accuracy_score #model evaluation
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

%matplotlib inline

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.7 MB/s eta 0:00:00
time: 357 µs (started: 2024-05-10 20:13:33 +00:00)


In [ ]:
print("hello world")

hello world
time: 533 µs (started: 2024-05-10 20:13:33 +00:00)


##This grabs skills, level, and networth of the players

In [ ]:
import requests

player_name = "deathstreeks"  # Replace with the player's Minecraft username

# Make a GET request to fetch the player's profile data
response = requests.get(f"https://sky.shiiyu.moe/api/v2/profile/{player_name}")

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    player_data = response.json()

    # Iterate over each profile to check if "current" is true
    for profile_uuid, profile_data in player_data.get('profiles', {}).items():
        if profile_data.get('current', False):
            # Extract the skill data for the specified player
            player_skills = {}

            # Navigate through the JSON structure to find the skill data
            if 'data' in profile_data and 'skills' in profile_data['data']:
                skills_data = profile_data['data']['skills'].get('skills', {})
                for skill_name in ['taming', 'farming', 'mining', 'combat', 'foraging', 'fishing', 'enchanting', 'alchemy', 'carpentry', 'runecrafting', 'social']:
                    skill_level = skills_data.get(skill_name, {}).get('level')
                    if skill_level is not None:
                        player_skills[skill_name] = skill_level

            # Extract player net worth
            player_networth = profile_data['data'].get('networth', {}).get('networth')

            # Extract player level
            player_level = profile_data['data'].get('skyblock_level', {}).get('level')

            # Print player data
            if player_level is not None:
                print(f"Player level: {player_level}")

            if player_networth is not None:
                print(f"Player net worth: {player_networth}")

            if player_skills:
                # Print the skill values for the player
                print("Player skills:")
                for skill_name, skill_level in player_skills.items():
                    print(f"{skill_name.capitalize()}: {skill_level}")

            if not player_skills and player_networth is None and player_level is None:
                print("No data found for the player.")

            # Exit the loop after processing the current profile
            break
    else:
        print("No current profile found for the player.")
else:
    # Print an error message if the request was unsuccessful
    print("Failed to fetch player data.")


Player level: 454
Player net worth: 325504102404.6822
Player skills:
Taming: 60
Farming: 60
Mining: 60
Combat: 60
Foraging: 50
Fishing: 50
Enchanting: 60
Alchemy: 50
Carpentry: 50
Runecrafting: 25
Social: 25


##This uses the list of player names and outputs level, networth, and skills, but instead of printing writes them to a new .csv file (warning: takes a LONG time to run)

In [ ]:
import csv
import requests

# Open the CSV file in write mode
with open('final_players_mode.csv', 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV writer
    fieldnames = ['Player Name', 'Level', 'Net Worth', 'Taming', 'Farming', 'Mining', 'Combat', 'Foraging', 'Fishing', 'Enchanting', 'Alchemy', 'Carpentry', 'Runecrafting', 'Social', 'Gamemode']
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    csv_writer.writeheader()

    # Open the text file containing player names
    with open('final_all_players.txt', 'r') as file:
        player_names = [line.strip() for line in file]

    # Iterate over each player name
    for player_name in player_names:
        # Make a GET request to fetch the player's profile data
        response = requests.get(f"https://sky.shiiyu.moe/api/v2/profile/{player_name}")

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            player_data = response.json()

            # Extract the current profile
            current_profile = None
            if 'profiles' in player_data:
                for profile_uuid, profile_data in player_data['profiles'].items():
                    if profile_data.get('current', False):
                        current_profile = profile_data
                        break

            if current_profile:
                # Extract player level
                player_level = current_profile['data'].get('skyblock_level', {}).get('level')

                # Extract player net worth
                player_networth = current_profile['data'].get('networth', {}).get('networth')

                # Extract player gamemode
                player_gamemode = current_profile.get('game_mode', '')

                # Extract player skills
                player_skills = {}
                if 'skills' in current_profile['data']:
                    skills_data = current_profile['data']['skills'].get('skills', {})
                    for skill_name in ['taming', 'farming', 'mining', 'combat', 'foraging', 'fishing', 'enchanting', 'alchemy', 'carpentry', 'runecrafting', 'social']:
                        skill_level = skills_data.get(skill_name, {}).get('level')
                        if skill_level is not None:
                            player_skills[skill_name.capitalize()] = skill_level

                # Write the player data to the CSV file
                row_data = {
                    'Player Name': player_name,
                    'Level': player_level,
                    'Net Worth': player_networth,
                    **player_skills,
                    'Gamemode': 'Ironman' if player_gamemode.lower() == 'ironman' else 'Not Ironman'
                }
                csv_writer.writerow(row_data)
            else:
                print(f"No current profile found for {player_name}.")
        else:
            # Print an error message if the request was unsuccessful
            print(f"Failed to fetch player data for {player_name}.")


Failed to fetch player data for EmeraldThrone12.
Failed to fetch player data for imanoub_.
Failed to fetch player data for xenstil_.
Failed to fetch player data for MaxUlysse.
Failed to fetch player data for videogarrett.
Failed to fetch player data for grayson9013.
Failed to fetch player data for Hiiroson.
Failed to fetch player data for derpo.
Failed to fetch player data for dbbo.
Failed to fetch player data for Draxyte.
Failed to fetch player data for DarkDPumpkin.
Failed to fetch player data for GeodeStrike.
Failed to fetch player data for strawstraw109.
Failed to fetch player data for the_dang_man69.
Failed to fetch player data for hydronium_core.
Failed to fetch player data for weenie.
Failed to fetch player data for IamThomas03.
Failed to fetch player data for Nerida_.
Failed to fetch player data for LavamanThePro.
Failed to fetch player data for Nightmare6178.
Failed to fetch player data for TreeDuck0716.
Failed to fetch player data for JahAllen.
Failed to fetch player data for

In [ ]:
df = pd.read_csv('final_1000_players.csv')

# Print the first few rows (head) of the DataFrame
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player Name   998 non-null    object 
 1   Level         998 non-null    int64  
 2   Net Worth     998 non-null    float64
 3   Taming        998 non-null    int64  
 4   Farming       998 non-null    int64  
 5   Mining        998 non-null    int64  
 6   Combat        998 non-null    int64  
 7   Foraging      998 non-null    int64  
 8   Fishing       998 non-null    int64  
 9   Enchanting    998 non-null    int64  
 10  Alchemy       998 non-null    int64  
 11  Carpentry     998 non-null    int64  
 12  Runecrafting  954 non-null    float64
 13  Social        954 non-null    float64
dtypes: float64(3), int64(10), object(1)
memory usage: 109.3+ KB


##The following paired down the top 1000 players to 100 and added in a bunch of random players from levels 1-400 in order to decrease the weight of the top players and add in an appropriate number of players below that level, so now I should have a big sample of players of all levels

In [ ]:
import csv

# Open the input and output CSV files
with open('top_1000_players.txt', 'r') as input_file, open('final_all_players.txt', 'w', newline='') as output_file:
    # Create a CSV reader and writer objects
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)

    # Loop through each row in the input CSV file
    for i, row in enumerate(reader):
        # Check if the row number is divisible by 4 (0-based index)
        if i % 10 == 0:
            # Write the selected row to the output CSV file
            writer.writerow(row)
    with open('random_players.txt', 'r') as random_file:
        for line in random_file:
            output_file.write(line)